# Inteligencia Artificial para generación en 2D

<a target="_blank" href="https://colab.research.google.com/github/griverat/Meteo-AI/blob/main/notebooks/5.gen_ai.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Si usa Google Colab, asegúrese de tener habilitada la GPU para este notebook.**

![gpu_colab](../images/colab_gpu.png)

## Descripción

En este notebook se presenta un ejemplo de cómo se puede utilizar una red neuronal convolucional para generar datos en 2D. Se explorará la generación de variables climáticas en 2D con datos de entrada en 2D. 

## Objetivos

- Comprender los conceptos básicos de las redes generativas.
- Explorar las distintas arquitecturas de redes neuronales convolucionales para la generación de campos en 2D.
- Desarrollar modelos de pronóstico con datos climáticos en 2D.

---

# Generación de datos en 2D

Previamente hemos explorado como una red neuronal puede tomar entradas tanto en 1D como en 2D para realizar predicciones en la forma de series temporales o clasificación. Estos casos presentaban una capa densa al final de la red para realizar la predicción. En este caso, la diferencia radica en que la red generativa no tiene una capa densa al final, sino que se utiliza una capa convolucional para generar la salida.

Los principales tipos de redes generativas son:

- **Generative Adversarial Networks (GANs)**: Son un tipo de red neuronal que se utiliza para generar datos nuevos. Consiste en dos redes neuronales, un generador y un discriminador, que compiten entre sí. El generador crea datos falsos y el discriminador intenta distinguir entre los datos reales y los falsos. El generador intenta engañar al discriminador y el discriminador intenta no ser engañado. Este proceso se repite hasta que el generador es capaz de generar datos que son indistinguibles de los datos reales.

- **Variational Autoencoders (VAEs)**: Son un tipo de red neuronal que se utiliza para generar datos nuevos. Consiste en dos redes neuronales, un codificador y un decodificador. El codificador toma una entrada y la convierte en un vector latente. El decodificador toma el vector latente y lo convierte en una salida. El objetivo es que el decodificador sea capaz de generar datos que sean indistinguibles de los datos reales.

En este notebook vamos a explorar algunos de los conceptos bases usados por estas arquitecturas ya que la finalidad es usar datos en 2D para generar datos en 2D. Las arquitecturas mencionadas anteriormente usualmente generan datos a partir de un vector latente que codifica la información de los datos de entrada, ya sea aleatorio o no.


### Importar librerías

In [ ]:
import os

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import xarray as xr
from IPython import display

plt.rcParams["font.family"] = "monospace"

Haremos uso del datos de reanálisis NCEP-NCAR 1 disponibles en la siguiente página web: https://psl.noaa.gov/data/gridded/data.ncep.reanalysis.html
El proyecto NCEP/NCAR Reanalysis 1 utiliza un sistema de análisis/pronóstico de última generación para realizar la asimilación de datos utilizando datos pasados ​​desde 1948 hasta el presente.

Para este ejemplo, utilizaremos los datos de temperatura y presión en superficie como predictores, y la precipitación como variable a predecir. Los datos que vamos a leer han sido cargados y recortados para una región específica sobre Perú con la finalidad de facilitar su uso en este notebook.

In [ ]:
# Esta celda contiene el codigo usado para el recorte y preprocesamiento de los datos
# puede ser reusado para la región de interés luego de descargar los datos de la
# pagina de NCEP
# temp = (
#     xr.open_mfdataset("/Users/dangomelon/Downloads/skin_temperature/*.nc")
#     .sel(time=slice("2020", None))
#     .skt.load()
#     - 273.15
# )
# temp = temp.sortby("lat").sel(lat=slice(-20, 10), lon=slice(278, 308))
# temp.to_netcdf("data/ncep_temp.nc")
# precip = (
#     xr.open_mfdataset("/Users/dangomelon/Downloads/prate/*.nc")
#     .sel(time=slice("2020", None))
#     .prate.load()
#     * 3600
#     * 24
# )
# precip = precip.sortby("lat").sel(lat=slice(-20, 10), lon=slice(278, 308))
# precip.to_netcdf("data/ncep_precip.nc")
# pres = (
#     xr.open_mfdataset("/Users/dangomelon/Downloads/pres/*.nc")
#     .sel(time=slice("2020", None))
#     .pres.load()
# )
# pres = pres.sortby("lat").sel(lat=slice(-20, 10), lon=slice(278, 308))
# pres.to_netcdf("data/ncep_pres.nc")

In [ ]:
!mkdir data
!wget https://raw.githubusercontent.com/griverat/Meteo-AI/main/notebooks/data/ncep_temp.nc -O data/ncep_temp.nc
!wget https://raw.githubusercontent.com/griverat/Meteo-AI/main/notebooks/data/ncep_precip.nc -O data/ncep_precip.nc
!wget https://raw.githubusercontent.com/griverat/Meteo-AI/main/notebooks/data/ncep_pres.nc -O data/ncep_pres.nc

In [ ]:
temp = xr.open_dataarray("data/ncep_temp.nc")
precip = xr.open_dataarray("data/ncep_precip.nc")
pres = xr.open_dataarray("data/ncep_pres.nc")

Exploramos el último dato de nuestras variables para tener una idea de cómo se ven los datos.

In [ ]:
fig, axs = plt.subplots(
    1,
    3,
    figsize=(15, 5),
    subplot_kw={"projection": ccrs.PlateCarree(central_longitude=180)},
)
axs = axs.flat
for i, (da, title) in enumerate(
    zip(
        [temp, pres, precip],
        ["Temperature", "Pressure", "Precipitation"],
    )
):
    ax = axs[i]
    da.isel(time=-1).plot(
        ax=ax, transform=ccrs.PlateCarree(), cmap="viridis", x="lon", y="lat"
    )
    ax.coastlines()
    ax.set_title(title)
    gl = ax.gridlines(draw_labels=True, ls="--", lw=0.5)
    gl.top_labels = False
    gl.right_labels = False

fig.tight_layout()

### Preparácion de los datos

Los datos tienen resolución temporal de 1 día, por lo que intentaremos usar los valores de los campos de temperatura y presión en superficie para predecir la precipitación en el siguiente día. Cabe notar que la elección de las variables es extremadamente importante al momento de diseñar un modelo de pronóstico, ya que no todas las variables son útiles para predecir una variable en particular. En este caso en particular, puede que la combinación de estas variables no sea la mejor para predecir la precipitación, pero su bajo peso y disponibilidad de datos en un solo nivel de presión facilita su uso en este ejemplo. En caso se quiera implementar una red con mejor precisión, se recomienda usar más variables en distintos niveles de presión.

Vamos a desfasar los datos de precipitation en 1

In [ ]:
precip_shifted = precip.shift(time=-1).dropna("time")
temp_data = temp.sel(time=precip_shifted.time)
pres_data = pres.sel(time=precip_shifted.time)

Ahora realizamos la separacion de los datos en conjuntos de entrenamiento, validación y prueba.

In [ ]:
train_size = int(0.8 * len(temp_data))
val_size = int(0.1 * len(temp_data))
test_size = len(temp_data) - train_size - val_size
print(f"Train size: {train_size}, Val size: {val_size}, Test size: {test_size}")

temp_train, temp_val, temp_test = (
    temp_data[:train_size],
    temp_data[train_size : train_size + val_size],
    temp_data[train_size + val_size :],
)
pres_train, pres_val, pres_test = (
    pres_data[:train_size],
    pres_data[train_size : train_size + val_size],
    pres_data[train_size + val_size :],
)

precip_train, precip_val, precip_test = (
    precip_shifted[:train_size],
    precip_shifted[train_size : train_size + val_size],
    precip_shifted[train_size + val_size :],
)

train_data = xr.concat([temp_train, pres_train], dim="channel").transpose(
    "time", "lat", "lon", "channel"
)

val_data = xr.concat([temp_val, pres_val], dim="channel").transpose(
    "time", "lat", "lon", "channel"
)

test_data = xr.concat([temp_test, pres_test], dim="channel").transpose(
    "time", "lat", "lon", "channel"
)

mean = train_data.mean(("time", "lat", "lon"))
std = train_data.std(("time", "lat", "lon"))

train_data = (train_data - mean) / std
val_data = (val_data - mean) / std
test_data = (test_data - mean) / std

Cabe resaltar que nuestros datos de entrada estan compuestos por 2 variables en 2D, por lo que las variables fueron combinadas en un solo tensor de entrada con la forma `(time, lat, lon, 2)`. De igual manera, se estandarizaron los datos de entrada debido al uso de variables con distintas escalas.

Haremos uso de la interfaz `Dataset` de tensorflow para cargar los datos con facilidad.

In [ ]:
train_data = tf.data.Dataset.zip(
    (
        tf.data.Dataset.from_tensor_slices(train_data.values),
        tf.data.Dataset.from_tensor_slices(precip_train.values),
    )
)

val_data = tf.data.Dataset.zip(
    (
        tf.data.Dataset.from_tensor_slices(val_data.values),
        tf.data.Dataset.from_tensor_slices(precip_val.values),
    )
)

Ahora que tenemos los datos organizados, vamos a comenzar definiendo una red neuronal convolucional como nuestra linea base para la generación de datos en 2D.


### Definición del modelo FCN

La red neuronal estara compuesta por una serie de capas convolucionales que mantienen la forma de nuestros datos de entrada y solo varían en la cantidad de filtros usados en cada capa. La salida de la red será un tensor de la misma forma que nuestros datos de entrada, pero con un solo canal de salida. Este tipo de red toma ventaja de la capacidad de las capas convolucionales para aprender patrones en 2D y generar datos en 2D.

In [ ]:
def get_fcn_model(input_shape):
    model_input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, 3, activation="relu", padding="same")(model_input)
    x = tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv2D(32, 3, activation="relu", padding="same")(x)

    model_output = tf.keras.layers.Conv2D(1, 3, activation="linear", padding="same")(x)

    # ya que la salida es precipitación, no puede ser negativa
    model_output = tf.keras.layers.ReLU()(model_output)

    model = tf.keras.Model(model_input, model_output)
    return model


fcn_model = get_fcn_model(tuple(train_data.element_spec[0].shape))
fcn_model.summary()

Se puede intentar variar algunos de los hiperparámetros de la red para ver si se obtienen mejores resultados. Por ejemplo, se puede intentar aumentar la cantidad de filtros en cada capa, o aumentar la cantidad de capas convolucionales. También se puede intentar modificar la cantidad de épocas de entrenamiento o el tamaño del lote.

In [ ]:
batch_size = 32

fcn_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.MeanSquaredError(),
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True, start_from_epoch=10
    )
]

history = fcn_model.fit(
    train_data.shuffle(len(train_data)).batch(batch_size),
    validation_data=val_data.batch(batch_size),
    epochs=100,
    callbacks=callbacks,
)

Revisamos las métricas de la red para ver cómo se comporta en el conjunto de prueba.

In [ ]:
metrics = pd.DataFrame(history.history)
metrics.plot()

Ahora evaluamos la red con el set de pruebas. Nuestros resultados pueden variar dependiendo de la cantidad de datos de entrenamiento y la cantidad de épocas que el modelo pudo entrenar antes de que el algoritmo de EarlyStopping detenga el entrenamiento.

In [ ]:
test_metrics = fcn_model.evaluate(test_data, precip_test)
print(f"Test MSE: {test_metrics:.2f}")

Colocamos la salida en un objeto de `xarray` que sea mas facil de manejar.

In [ ]:
preds = fcn_model.predict(test_data)
preds = xr.DataArray(
    preds.squeeze(),
    dims=["time", "lat", "lon"],
    coords={
        "time": test_data.time,
        "lat": test_data.lat,
        "lon": test_data.lon,
    },
)

Realizamos un gráfico con 3 paneles para comparar los datos reales con los datos generados y el error entre ellos. De esta forma podemos ver en que ubicaciones la red ha sobreestimado o subestimado la precipitación.

In [ ]:
idx = np.random.randint(0, len(test_data))

fig, axs = plt.subplots(
    1,
    3,
    figsize=(15, 5),
    subplot_kw={"projection": ccrs.PlateCarree(central_longitude=180)},
)
axs = axs.flat
precip_test.isel(time=idx).plot(
    ax=axs[0], transform=ccrs.PlateCarree(), cmap="viridis", vmax=20
)
preds.isel(time=idx).plot(
    ax=axs[1], transform=ccrs.PlateCarree(), cmap="viridis", vmax=20
)
(preds - precip_test).isel(time=idx).plot(
    ax=axs[2],
    transform=ccrs.PlateCarree(),
    cmap="coolwarm",
    robust=True,
    vmax=10,
    vmin=-10,
)

for ax in axs:
    ax.coastlines()
    gl = ax.gridlines(draw_labels=True, ls="--", lw=0.5, alpha=0.5)
    gl.top_labels = False
    gl.right_labels = False

## Arquitectura U-Net

La arquitectura U-Net es una red neuronal convolucional que se utiliza para la segmentación de imágenes. La red toma una imagen de entrada y produce una imagen de salida que contiene la segmentación de la imagen de entrada. La red consta de dos partes, un codificador y un decodificador. El codificador toma la imagen de entrada y la reduce a una representación latente. El decodificador toma la representación latente y la expande a la imagen de salida. La red utiliza conexiones residuales para ayudar a la propagación del gradiente durante el entrenamiento.

A diferenca de la red FCN, la red U-Net tiene una arquitectura más compleja que permite aprender patrones en distintas escalas de la imagen. La red U-Net es una red profunda que consta de varias capas convolucionales y de pooling en el codificador, y varias capas convolucionales y de upsampling en el decodificador. Usualmente hacen uso de conexiones residuales para ayudar a la propagación del gradiente durante el entrenamiento.

En este ejemplo vamos a implementar una red U-Net para la generación de datos en 2D sin conexiones residuales. 

In [ ]:
def get_unet_model(input_shape=(16, 16, 1)):
    model_input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, 3, activation="relu", padding="same")(model_input)
    x = tf.keras.layers.Conv2D(32, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.MaxPool2D()(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.MaxPool2D()(x)

    x = tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.MaxPool2D()(x)

    bottleneck = tf.keras.layers.Conv2D(256, 3, activation="relu", padding="same")(x)

    x = tf.keras.layers.Conv2DTranspose(
        128, 3, strides=2, activation="relu", padding="same"
    )(bottleneck)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same")(x)

    x = tf.keras.layers.Conv2DTranspose(
        64, 3, strides=2, activation="relu", padding="same"
    )(x)
    x = tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same")(x)

    x = tf.keras.layers.Conv2DTranspose(
        32, 3, strides=2, activation="relu", padding="same"
    )(x)
    x = tf.keras.layers.Conv2D(32, 3, activation="relu", padding="same")(x)

    model_output = tf.keras.layers.Conv2D(1, 3, activation="linear", padding="same")(x)

    # ya que la salida es precipitación, no puede ser negativa
    model_output = tf.keras.layers.ReLU()(model_output)

    model = tf.keras.Model(model_input, model_output)
    return model


unet_model = get_unet_model(tuple(train_data.element_spec[0].shape))
unet_model.summary()

Ahora seguimos con el código para el entrenamiento de la red.

In [ ]:
batch_size = 64

unet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.MeanSquaredError(),
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True, start_from_epoch=10
    )
]

history = unet_model.fit(
    train_data.shuffle(len(train_data)).batch(batch_size),
    validation_data=val_data.batch(batch_size),
    epochs=100,
    callbacks=callbacks,
)

Nuevamente, esto nos puede generar resultados variados, todo depende de la inicialización aleatoria de los pesos de la red. En caso de que los resultados no sean los esperados, se puede intentar cambiar algunos hiperparámetros de la red, como la cantidad de filtros en cada capa, la cantidad de capas convolucionales, la cantidad de épocas de entrenamiento, o el tamaño del lote.

In [ ]:
metrics = pd.DataFrame(history.history)
metrics.plot(ylim=(0, 150))

In [ ]:
preds = unet_model.predict(test_data)
preds = xr.DataArray(
    preds.squeeze(),
    dims=["time", "lat", "lon"],
    coords={
        "time": test_data.time,
        "lat": test_data.lat,
        "lon": test_data.lon,
    },
)

In [ ]:
idx = np.random.randint(0, len(test_data))

fig, axs = plt.subplots(
    1,
    3,
    figsize=(15, 5),
    subplot_kw={"projection": ccrs.PlateCarree(central_longitude=180)},
)
axs = axs.flat
precip_test.isel(time=idx).plot(
    ax=axs[0], transform=ccrs.PlateCarree(), cmap="viridis", vmax=20
)
preds.isel(time=idx).plot(
    ax=axs[1], transform=ccrs.PlateCarree(), cmap="viridis", vmax=20
)
(preds - precip_test).isel(time=idx).plot(
    ax=axs[2],
    transform=ccrs.PlateCarree(),
    cmap="coolwarm",
    robust=True,
    vmax=10,
    vmin=-10,
)

for ax in axs:
    ax.coastlines()
    gl = ax.gridlines(draw_labels=True, ls="--", lw=0.5, alpha=0.5)
    gl.top_labels = False
    gl.right_labels = False

### U-Net con conexiones residuales

La red U-Net con conexiones residuales es una variante de la red U-Net que ayuda a la propagación del gradiente durante el entrenamiento. La red consta de dos partes, un codificador y un decodificador. El codificador toma la imagen de entrada y la reduce a una representación latente. El decodificador toma la representación latente y la expande a la imagen de salida. Las conexiones residuales conectan las capas del codificador con las capas del decodificador para ayudar de esta forma a la propagación del gradiente y mejorar el rendimiento de la red.

Ya que esta red se compone de varias secuencias de capas convolucionales y de pooling en el codificador, y capas convolucionales y de upsampling en el decodificador, usaremos funciones que nos ayuden a definir estas capas de forma más sencilla.

In [ ]:
def double_conv_block(x, n_filters):

    # Conv2D luego ReLU
    x = tf.keras.layers.Conv2D(n_filters, 3, padding="same", activation="relu")(x)
    # Conv2D luego ReLU
    x = tf.keras.layers.Conv2D(n_filters, 3, padding="same", activation="relu")(x)

    return x


def downsample_block(x, n_filters):
    f = double_conv_block(x, n_filters)
    p = tf.keras.layers.MaxPool2D(2)(f)
    p = tf.keras.layers.Dropout(0.3)(p)

    return f, p


def upsample_block(x, conv_features, n_filters):
    # upsample
    x = tf.keras.layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
    # concatenar con la salida de la capa de downsample
    # aca es donde se realiza la conexion residual
    x = tf.keras.layers.concatenate([x, conv_features])
    # dropout
    x = tf.keras.layers.Dropout(0.3)(x)
    # Conv2D luego ReLU
    x = double_conv_block(x, n_filters)

    return x


def unet_res():
    # entrada
    inputs = tf.keras.layers.Input(shape=(16, 16, 2))

    # encoder: camino de reducción - downsample
    # 1 - downsample
    f1, p1 = downsample_block(inputs, 64)
    # 2 - downsample
    f2, p2 = downsample_block(p1, 128)
    # 3 - downsample
    f3, p3 = downsample_block(p2, 256)
    # 4 - downsample
    f4, p4 = downsample_block(p3, 512)

    # 5 - bottleneck - cuello de botella - espacio latente
    bottleneck = double_conv_block(p4, 1024)

    # decoder: camino de expansión - upsample
    # 6 - upsample
    u6 = upsample_block(bottleneck, f4, 512)
    # 7 - upsample
    u7 = upsample_block(u6, f3, 256)
    # 8 - upsample
    u8 = upsample_block(u7, f2, 128)
    # 9 - upsample
    u9 = upsample_block(u8, f1, 64)

    # salidas
    outputs = tf.keras.layers.Conv2D(1, 1, padding="same", activation="relu")(u9)

    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")

    return unet_model

Vamos a explorar la arquitectura de la red U-Net con conexiones residuales.

In [ ]:
generator = unet_res()
generator.summary()

Keras provee una forma gráfica de visualizar la arquitectura de la red. Esto nos puede ayudar a entender mejor cómo se conectan las capas de la red y cómo se propagan los datos a través de la red.

In [ ]:
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

Debido a la complejidad del modelo, el entrenamiento puede tardar más tiempo que los modelos anteriores. Se recomienda tener paciencia y dejar que el modelo se entrene por un número suficiente de épocas.

In [ ]:
batch_size = 64

generator.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.MeanSquaredError(),
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True, start_from_epoch=10
    )
]

history = generator.fit(
    train_data.shuffle(len(train_data)).batch(batch_size),
    validation_data=val_data.batch(batch_size),
    epochs=100,
    callbacks=callbacks,
)

In [ ]:
metrics = pd.DataFrame(history.history)
metrics.plot()

In [ ]:
preds = generator.predict(test_data)
preds = xr.DataArray(
    preds.squeeze(),
    dims=["time", "lat", "lon"],
    coords={
        "time": test_data.time,
        "lat": test_data.lat,
        "lon": test_data.lon,
    },
)

In [ ]:
idx = np.random.randint(0, len(test_data))

fig, axs = plt.subplots(
    1,
    3,
    figsize=(15, 5),
    subplot_kw={"projection": ccrs.PlateCarree(central_longitude=180)},
)
axs = axs.flat
precip_test.isel(time=idx).plot(
    ax=axs[0], transform=ccrs.PlateCarree(), cmap="viridis", vmax=20
)
preds.isel(time=idx).plot(
    ax=axs[1], transform=ccrs.PlateCarree(), cmap="viridis", vmax=20
)
(preds - precip_test).isel(time=idx).plot(
    ax=axs[2],
    transform=ccrs.PlateCarree(),
    cmap="coolwarm",
    robust=True,
    vmax=10,
    vmin=-10,
)

for ax in axs:
    ax.coastlines()
    gl = ax.gridlines(draw_labels=True, ls="--", lw=0.5, alpha=0.5)
    gl.top_labels = False
    gl.right_labels = False